In [ ]:
import os

import dgl
import torch
import torch_geometric as tg

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import dgl.nn.pytorch as dglnn

from graphlime import GraphLIME
from mumin import MuminDataset

import torchmetrics as tm

from tqdm.notebook import tqdm
from collections import defaultdict

from torch_geometric.data import Data as tgData

from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

from mumin_explainable.architectures.graphs import GAT
from mumin_explainable.processor.tweetnormalizer import normalizeTweet

from dotenv import load_dotenv

%matplotlib inline

_= torch.manual_seed(42)

# Setup mumin graph

In [ ]:
size= 'small'

dataset_mumin = MuminDataset(
    twitter_bearer_token=os.getenv('TWITTER_BEARER_TOKEN'),
    size=size,
    dataset_path=f'./data/datasets/mumin-{size}.zip'
)
dataset_mumin.compile()
mumin_graph = dataset_mumin.to_dgl()
mumin_graph

In [ ]:
plt.figure(figsize=(10, 7))
metagraph = mumin_graph.metagraph()
nx.draw_networkx(metagraph, 
                 pos=nx.shell_layout(metagraph), 
                 node_color='white', 
                 node_size=3000,
                 arrows=False)

In [ ]:
claim_df = dataset_mumin.nodes['claim']
tweet_df = dataset_mumin.nodes['tweet']

tweet_discusses_claim_df = dataset_mumin.rels[('tweet', 'discusses', 'claim')]

# Setup subgraph

In [ ]:
tweet_discusses_claim_subgraph = dgl.edge_type_subgraph(mumin_graph, etypes=[('tweet', 'discusses', 'claim')])

## Filter language

In [ ]:
LANG = 'multilingual'
claim_ds_copy = dataset_mumin.nodes['claim'].dropna()
lang_claim = (claim_ds_copy['language'] == LANG).to_list()
del(claim_ds_copy)

In [ ]:
if LANG == 'multilingual':
    lang_claim = [True] * len(lang_claim)
claim_train_mask = tweet_discusses_claim_subgraph.nodes['claim'].data['train_mask'] & torch.tensor(lang_claim)
claim_val_mask = tweet_discusses_claim_subgraph.nodes['claim'].data['val_mask'] & torch.tensor(lang_claim)
claim_test_mask = tweet_discusses_claim_subgraph.nodes['claim'].data['test_mask'] & torch.tensor(lang_claim)

plt.figure(figsize=(10, 7))
tweet_discusses_claim_metagraph = tweet_discusses_claim_subgraph.metagraph()
nx.draw_networkx(tweet_discusses_claim_metagraph, 
                 pos=nx.shell_layout(tweet_discusses_claim_metagraph), 
                 node_color='white', 
                 node_size=3000,
                 arrows=False)

# Setup dataset

In [ ]:
edges_index = torch.cat([
    tweet_discusses_claim_subgraph.edges(etype='discusses')[0].unsqueeze(0),
    tweet_discusses_claim_subgraph.edges(etype='discusses')[1].unsqueeze(0)
], dim=0)

_claim_x = torch.stack([tweet_discusses_claim_subgraph.nodes['claim'].data['feat'][i] for i in tweet_discusses_claim_subgraph.edges(etype='discusses')[1]])
_claim_y = torch.stack([tweet_discusses_claim_subgraph.nodes['claim'].data['label'][i] for i in tweet_discusses_claim_subgraph.edges(etype='discusses')[1]])
_claim_train_mask = torch.BoolTensor([claim_train_mask[i] for i in tweet_discusses_claim_subgraph.edges(etype='discusses')[1]])
_claim_val_mask = torch.BoolTensor([claim_val_mask[i] for i in tweet_discusses_claim_subgraph.edges(etype='discusses')[1]])
_claim_test_mask = torch.BoolTensor([claim_test_mask[i] for i in tweet_discusses_claim_subgraph.edges(etype='discusses')[1]])
data = tgData(
    x=_claim_x,
    y=_claim_y,
    train_mask=_claim_train_mask,
    val_mask=_claim_val_mask,
    test_mask=_claim_test_mask,
    edge_index=edges_index.long())

# Train GAT

In [ ]:
MODEL_NAME = f'{LANG}_graph'
hparams = {
    'input_dim': data.num_node_features,
    'hidden_dim': 16,
    'output_dim': max(data.y).item() + 1
}

model = GAT(**hparams).double()

lr = 0.005
epochs = 400

model.train()
optimizer = optim.Adam(model.parameters(), lr=lr)

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
stats_score = tm.StatScores(task='multiclass', num_classes=2, average='none')
precision = tm.Precision(task='multiclass', num_classes=2, average='none')
recall = tm.Recall(task='multiclass', num_classes=2, average='none')
f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')
best_f1macro = -1

bootstrap = tm.BootStrapper(
    f1_score, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

for epoch in tqdm(range(epochs)):
    optimizer.zero_grad()
    
    output = model(data.x, data.edge_index)
    loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
    
    loss.backward()
    optimizer.step()
    
    f1 = f1_score(output[data.train_mask], data.y[data.train_mask])
    f1macro = torch.mean(f1)

    bootstrap.update(output[data.train_mask], data.y[data.train_mask])
    
    if f1macro > best_f1macro:
        best_f1macro = f1macro
        torch.save(model.state_dict(), f'./data/models/{MODEL_NAME}.pth')

    if epoch % 10 == 0:
        metrics = {
            'Epoch': epoch,
            'Accuracy': accuracy(output[data.train_mask], data.y[data.train_mask]),
            'Precision': precision(output[data.train_mask], data.y[data.train_mask]),
            'Recall': recall(output[data.train_mask], data.y[data.train_mask]),
            'Stats_Score': stats_score(output[data.train_mask], data.y[data.train_mask]),
            'F1': f1,
            'F1-macro': f1macro,
            'Bootstrap': bootstrap.compute()
        }
        print(metrics)

In [ ]:
model.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model.eval()

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
stats_score = tm.StatScores(task='multiclass', num_classes=2, average='none')
precision = tm.Precision(task='multiclass', num_classes=2, average='none')
recall = tm.Recall(task='multiclass', num_classes=2, average='none')
f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')

print('test')

f1 = f1_score(output[data.test_mask], data.y[data.test_mask])
f1macro = torch.mean(f1)

metrics = {
    'Accuracy': accuracy(output[data.test_mask], data.y[data.test_mask]),
    'Precision': precision(output[data.test_mask], data.y[data.test_mask]),
    'Recall': recall(output[data.test_mask], data.y[data.test_mask]),
    'Stats_Score': stats_score(output[data.test_mask], data.y[data.test_mask]),
    'F1': f1,
    'F1-macro': f1macro,
    'Bootstrap': bootstrap.compute()
}

print(metrics)

In [ ]:
model.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model.eval()

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
acc_bs = tm.BootStrapper(
    accuracy, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')
f1_bs = tm.BootStrapper(
    f1_score, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

acc_bs.update(output[data.test_mask], data.y[data.test_mask])
f1_bs.update(output[data.test_mask], data.y[data.test_mask])

out_acc_bs = acc_bs.compute()
out_f1_bs = f1_bs.compute()

results = {
    'Language': LANG,
    'Accuracy Bootstrap': out_acc_bs,
    'F1 Bootstrap': out_f1_bs
}

print(results)

In [ ]:
def get_all_nodes_explanation(model, explainer, data, dataset_mumin):

    feature_id_map = {
        'num_retweets': 0,
        'num_replies': 1,
        'num_quote_tweets': 2
    }

    user_df = dataset_mumin.nodes['user']
    tweet_df = dataset_mumin.nodes['tweet']
    reply_df = dataset_mumin.nodes['reply']

    reply_quoteof_tweet_df = dataset_mumin.rels[('reply', 'quote_of', 'tweet')]
    reply_quoteof_tweet_df = (reply_df.merge(reply_quoteof_tweet_df, left_index=True, right_on='src')
                            .merge(tweet_df, left_on='tgt', right_index=True)
                            .reset_index(drop=True))

    user_posted_tweet_df = dataset_mumin.rels[('user', 'posted', 'tweet')]
    user_posted_tweet_df = (user_df.merge(user_posted_tweet_df, left_index=True, right_on='src')
                            .merge(tweet_df, left_on='tgt', right_index=True)
                            .reset_index(drop=True))

    user_posted_reply_df = dataset_mumin.rels[('user', 'posted', 'reply')]
    user_posted_reply_df = (user_df.merge(user_posted_reply_df, left_index=True, right_on='src')
                            .merge(tweet_df, left_on='tgt', right_index=True)
                            .reset_index(drop=True))

    for node_idx in range(data.x.shape[0]):
        
        coefs = explainer.explain_node(node_idx, data.x, data.edge_index)
        probas = model(data.x, data.edge_index).exp()
        # fact_or_fake = 'fact' if torch.argmax(probas[node_idx]).item() == 0 else 'fake'

        if tweet_df.iloc[node_idx]['num_quote_tweets'] != 0 and \
            feature_id_map['num_quote_tweets'] not in list(np.where(coefs != 0)[0]): # get only inferences explained by quotes

            tgt_tweet_id = dataset_mumin.nodes['tweet'].iloc[node_idx]['tweet_id']

            # manual traverse
            replies_src = list(reply_quoteof_tweet_df.query(f'tweet_id_y == {tgt_tweet_id}')['src'])                      
            quoters_ids = list(user_posted_reply_df.query(f'tgt in {str(replies_src)}')['user_id'])
            quoters_posts = user_posted_tweet_df[user_posted_tweet_df['user_id'].isin(quoters_ids)]

            if not quoters_posts.empty:
                # print(fact_or_fake, quoters_ids)
                print(quoters_ids)
                # break

In [ ]:
# user_df = dataset_mumin.nodes['user']
# explainer = GraphLIME(model, hop=2, rho=0.1, cached=True)
# get_all_nodes_explanation(model, explainer, data, dataset_mumin)

# Enhance with text-based features

In [ ]:
from transformers import AutoModelForSequenceClassification

TEXT_DIM = 100

LANG_TOOL_MAP = {
    'multilingual': {
        'bertweet': AutoModel.from_pretrained('vinai/bertweet-base'),
        'tokenizer': AutoTokenizer.from_pretrained('vinai/bertweet-base', use_fast=False)
    },
    'en': {
        'bertweet': AutoModel.from_pretrained('vinai/bertweet-base'),
        'tokenizer': AutoTokenizer.from_pretrained('vinai/bertweet-base', use_fast=False)
    },
    'pt': {
        'bertweet': AutoModel.from_pretrained('melll-uff/bertweetbr'),
        'tokenizer': AutoTokenizer.from_pretrained('melll-uff/bertweetbr', normalization=True)
    },
    'es': {
        'bertweet': AutoModel.from_pretrained('pysentimiento/robertuito-base-cased'),
        'tokenizer': AutoTokenizer.from_pretrained('pysentimiento/robertuito-base-cased')
    }
}

bertweet = LANG_TOOL_MAP[LANG]['bertweet']
tokenizer = LANG_TOOL_MAP[LANG]['tokenizer']

# bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)
# bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)

In [ ]:
def tweetencoder(x, text_dim):
    try:
        x = bertweet(torch.tensor([tokenizer.encode(normalizeTweet(x))])).pooler_output
    except:
        # print(x)
        x = bertweet(torch.tensor([tokenizer.encode('')])).pooler_output
    return nn.Linear(768, text_dim)(x).tolist()[0]

In [ ]:
tweet_df['text_encoding'] = str([0] * TEXT_DIM)
if LANG == 'multilingual':
    tweet_df['text_encoding'] = [tweetencoder(text, TEXT_DIM) for text in tweet_df['text']]
else:
    tweet_df['text_encoding'] = [tweetencoder(text, TEXT_DIM) if lang == LANG else str([0] * TEXT_DIM) for text,lang in zip(tweet_df['text'], tweet_df['lang'])]

In [ ]:
new_embedding_columns = [f'emb{i}' for i in range(TEXT_DIM)]

tweet_embeddings_split_df = pd.DataFrame(
    [x if not isinstance(x, str) else eval(x) for x in tweet_df['text_encoding'].tolist()],
    index=tweet_df.index,
    columns=new_embedding_columns
)
tweet_df = pd.concat([tweet_df, tweet_embeddings_split_df], axis=1)
tweet_df.dropna(inplace=True)
display(tweet_df)

In [ ]:
MODE = 'multimodal'
# MODE = 'text'

new_features_df = pd.DataFrame(index=range(tweet_discusses_claim_subgraph.nodes['tweet'].data['feat'].shape[0]))
new_features_df = new_features_df.join(tweet_df[new_embedding_columns])#.fillna(0)
new_features_tensor = torch.tensor(new_features_df.values).double()

mixed_features = new_features_tensor
if MODE == 'multimodal':
    mixed_features = torch.cat([tweet_discusses_claim_subgraph.nodes['tweet'].data['feat'], new_features_tensor], axis = 1).double()

edges_index = torch.cat([
    tweet_discusses_claim_subgraph.edges(etype='discusses')[0].unsqueeze(0),
    tweet_discusses_claim_subgraph.edges(etype='discusses')[1].unsqueeze(0)
], dim=0)

_claim_x = torch.stack([mixed_features[i] for i in tweet_discusses_claim_subgraph.edges(etype='discusses')[1]])
_claim_y = torch.stack([tweet_discusses_claim_subgraph.nodes['claim'].data['label'][i] for i in tweet_discusses_claim_subgraph.edges(etype='discusses')[1]])
_claim_train_mask = torch.BoolTensor([claim_train_mask[i] for i in tweet_discusses_claim_subgraph.edges(etype='discusses')[1]])
_claim_val_mask = torch.BoolTensor([claim_val_mask[i] for i in tweet_discusses_claim_subgraph.edges(etype='discusses')[1]])
_claim_test_mask = torch.BoolTensor([claim_test_mask[i] for i in tweet_discusses_claim_subgraph.edges(etype='discusses')[1]])

new_features_data = tgData(
    x=_claim_x,
    y=_claim_y,
    train_mask=_claim_train_mask,
    val_mask=_claim_val_mask,
    test_mask=_claim_test_mask,
    edge_index=edges_index.long())

In [ ]:
hparams = {
    'input_dim': new_features_data.num_node_features,
    'hidden_dim': 16,
    'output_dim': max(new_features_data.y).item() + 1
}

MODEL_NAME = f'{LANG}_{MODE}'

model = GAT(**hparams).double()

lr = 0.005
epochs = 400

model.train()
optimizer = optim.Adam(model.parameters(), lr=lr)

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
stats_score = tm.StatScores(task='multiclass', num_classes=2, average='none')
precision = tm.Precision(task='multiclass', num_classes=2, average='none')
recall = tm.Recall(task='multiclass', num_classes=2, average='none')
f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')
best_f1macro = -1

bootstrap = tm.BootStrapper(
    f1_score, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

for epoch in tqdm(range(epochs)):
    optimizer.zero_grad()
    
    output = model(new_features_data.x, new_features_data.edge_index)
    loss = F.nll_loss(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask])
    
    loss.backward()
    optimizer.step()
    
    f1 = f1_score(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask])
    f1macro = torch.mean(f1)

    bootstrap.update(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask])

    if f1macro > best_f1macro:
        best_f1macro = f1macro
        torch.save(model.state_dict(), f'./data/models/{MODEL_NAME}.pth')

    if epoch % 10 == 0:
        metrics = {
            'Epoch': epoch,
            'Accuracy': accuracy(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask]),
            'Precision': precision(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask]),
            'Recall': recall(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask]),
            'Stats_Score': stats_score(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask]),
            'F1': f1,
            'F1-macro': f1macro,
            'Bootstrap': bootstrap.compute()
        }
        print(metrics)

model.eval()

In [ ]:
model.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model.eval()

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
stats_score = tm.StatScores(task='multiclass', num_classes=2, average='none')
precision = tm.Precision(task='multiclass', num_classes=2, average='none')
recall = tm.Recall(task='multiclass', num_classes=2, average='none')
f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')

print('test')

f1 = f1_score(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask])
f1macro = torch.mean(f1)

metrics = {
    'Accuracy': accuracy(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask]),
    'Precision': precision(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask]),
    'Recall': recall(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask]),
    'Stats_Score': stats_score(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask]),
    'F1': f1,
    'F1-macro': f1macro,
    'Bootstrap': bootstrap.compute()
}

print(metrics)

In [ ]:
model.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model.eval()

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
acc_bs = tm.BootStrapper(
    accuracy, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')
f1_bs = tm.BootStrapper(
    f1_score, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

acc_bs.update(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask])
f1_bs.update(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask])

out_acc_bs = acc_bs.compute()
out_f1_bs = f1_bs.compute()

results = {
    'Language': LANG,
    'Accuracy Bootstrap': out_acc_bs,
    'F1 Bootstrap': out_f1_bs
}

print(results)

In [ ]:
# list(tweet_df[tweet_df['lang'] == 'pt'].index)

In [ ]:
explainer = GraphLIME(model, hop=2, rho=0.1, cached=True)

# for node_idx in range(data.x.shape[0]):
#     coefs = explainer.explain_node(node_idx, data.x, data.edge_index)
#     if len(set(np.where(coefs != 0)[0]).intersection(set([3,4,5]))) != 0:
#         print(node_idx)

# try: 102 | 118 | 127
node_idx = 91

probas = model(new_features_data.x, new_features_data.edge_index).exp()
print(probas[node_idx], torch.argmax(probas[node_idx]).item())
coefs = explainer.explain_node(node_idx, new_features_data.x, new_features_data.edge_index)

print(coefs, len(coefs))
print(np.where(coefs != 0))

In [ ]:
k = 91
print(tweet_df.loc[k].text)
tweet_df.loc[k]

In [ ]:
tweet_df = dataset_mumin.nodes['tweet']
claim_df = dataset_mumin.nodes['claim']
x = dataset_mumin.rels[('tweet', 'discusses', 'claim')]
y = (tweet_df.merge(x, left_index=True, right_on='src')
                          .merge(claim_df, left_on='tgt', right_index=True)
                          .reset_index(drop=True))

y[y['tweet_id'] == 1334273990039375876]

In [ ]:
tweet_df = dataset_mumin.nodes['tweet']
reply_df = dataset_mumin.nodes['reply']
reply_quoteof_tweet_df = dataset_mumin.rels[('reply', 'reply_to', 'tweet')]
reply_quoteof_tweet_df = (reply_df.merge(quote_of_df, left_index=True, right_on='src')
                        .merge(tweet_df, left_on='tgt', right_index=True)
                        .reset_index(drop=True))

reply_quoteof_tweet_df[reply_quoteof_tweet_df['tweet_id_y'] == 1334273990039375876]

In [ ]:
from transformers_interpret import SequenceClassificationExplainer, MultiLabelClassificationExplainer
# cls_explainer = SequenceClassificationExplainer(bertweet,tokenizer)
cls_explainer = SequenceClassificationExplainer(bertweet,tokenizer)

In [ ]:
word_attributions = cls_explainer(normalizeTweet(tweet_df.loc[127].text))

In [ ]:
cls_explainer.predicted_class_name

In [ ]:
cls_explainer.visualize()

In [ ]:
word_attributions = cls_explainer(normalizeTweet('Head of Pfizer Research: Covid Vaccine is Female Sterilization – Health and Money News https://t.co/IDRLSVmkLz'))

In [ ]:
cls_explainer.visualize()

In [ ]:
bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)
F.softmax(bertweet(torch.tensor([tokenizer.encode(normalizeTweet('my favourite text'))])).logits)

In [ ]:
F.softmax(bertweet(torch.tensor([tokenizer.encode(normalizeTweet('my favourite text'))])).logits, dim=1)

In [ ]:
cls_explainer = SequenceClassificationExplainer(bertweet,tokenizer)

In [ ]:
word_attributions = cls_explainer(normalizeTweet(tweet_df.loc[102].text))
cls_explainer.visualize()

In [ ]:
dataset_mumin.nodes['claim'].loc[0]['embedding'].shape

In [ ]:
dataset_mumin.nodes['tweet']

In [ ]:
bertweet.config.id2label

In [ ]:
tweet_discusses_claim_df

In [ ]:
dataset_mumin.nodes['claim']